In [2]:
import pandas as pd
import pyodbc

DB = {
    'servername': 'JULLAPTAB', 
    'database': 'UnitedOutdoors'
}


In [3]:
export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'])
export_cursor = export_conn.cursor()
export_cursor


OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server does not exist or access denied. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (53)')

In [ ]:
conn_str_access = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\Users\julia\OneDrive\Desktop\THUAS\ICT\Year2\Semester 4\Project\UO_bedrijf\aenc.accdb;'
)
aec_conn = pyodbc.connect(conn_str_access)


    ProductID                      ProductName  SupplierID  CategoryID  \
0           1                             Chai           1           1   
1           2                            Chang           1           1   
2           3                    Aniseed Syrup           1           2   
3           4     Chef Anton's Cajun Seasoning           2           2   
4           5           Chef Anton's Gumbo Mix           2           2   
..        ...                              ...         ...         ...   
72         73                       Röd Kaviar          17           8   
73         74                    Longlife Tofu           4           7   
74         75             Rhönbräu Klosterbier          12           1   
75         76                     Lakkalikööri          23           1   
76         77  Original Frankfurter grüne Soße          12           2   

        QuantityPerUnit  UnitPrice  UnitsInStock  UnitsOnOrder  ReorderLevel  \
0    10 boxes x 20 bags      18

C:\Users\antho\AppData\Local\Temp\ipykernel_16600\1704179019.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  products_df = pd.read_sql_query(products_query, export_conn)


In [ ]:
northwind_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=Northwind')
adventureworks_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=AdventureWorks2019')


    id          name        description          prod_size   color quantity  \
0  300     Tee Shirt           Tank Top              Small   White       18   
1  301     Tee Shirt             V-neck             Medium  Orange       39   
2  302     Tee Shirt          Crew Neck  One size fits all   Black       72   
3  400  Baseball Cap         Cotton Cap  One size fits all   Black       92   
4  401  Baseball Cap           Wool cap  One size fits all   White       12   
5  500         Visor        Cloth Visor  One size fits all   White       36   
6  501         Visor      Plastic Visor  One size fits all   Black       28   
7  600    Sweatshirt  Hooded Sweatshirt              Large   Green       39   
8  601    Sweatshirt  Zipped Sweatshirt              Large    Blue       32   
9  700        Shorts      Cotton Shorts             Medium   Black       80   

  unit_price picture_name     Category  
0          9  tshirtw.bmp      Clothes  
1         14  tshirto.bmp      Clothes  
2      

C:\Users\antho\AppData\Local\Temp\ipykernel_16600\2144621771.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product_df = pd.read_sql_query(product_query, aec_conn)


In [ ]:
products_query = """
SELECT NULL AS NewProductID, P.ProductID, P.ProductName, C.Description AS ProductDescription, C.CategoryName AS ProductCategory, 
       NULL AS Color, NULL AS Size, NULL AS ProductQuantity, P.QuantityPerUnit, CAST(P.UnitPrice AS FLOAT) AS UnitPrice, NULL AS ListPrice, P.UnitsInStock 
FROM Products P 
JOIN Categories C ON P.CategoryID = C.CategoryID
"""
products_df = pd.read_sql_query(products_query, northwind_conn)
products_df['Source'] = 'Northwind'
print(products_df)


In [ ]:
product_query = """
SELECT NULL AS NewProductID, id AS ProductID, name AS ProductName, description AS ProductDescription, Category AS ProductCategory, 
       color AS ProductColor, prod_size AS ProductSize, quantity AS ProductQuantity, NULL AS QuantityPerUnit, unit_price AS UnitPrice, 
       NULL AS ListPrice, NULL AS UnitsInStock
FROM product
"""
product_df = pd.read_sql_query(product_query, aec_conn)
product_df['Source'] = 'AenC'

# Converteer UnitPrice kolom naar float
product_df['UnitPrice'] = product_df['UnitPrice'].astype(float)
print(product_df)


In [ ]:
new_product_query = """
SELECT NULL AS NewProductID, P.ProductID, P.Name AS ProductName, S.Name AS ProductDescription, C.Name AS ProductCategory, P.Color AS ProductColor, P.Size AS ProductSize, NULL AS ProductQuantity, 
       NULL AS QuantityPerUnit, CAST(P.StandardCost AS FLOAT) AS UnitPrice, CAST(P.ListPrice AS FLOAT) AS ListPrice, NULL AS UnitsInStock
FROM Production.Product P
JOIN Production.ProductSubcategory S
ON p.ProductSubcategoryID = S.ProductSubcategoryID
JOIN Production.ProductCategory C 
ON S.ProductCategoryID = C.ProductCategoryID
"""
new_product_df = pd.read_sql_query(new_product_query, adventureworks_conn)
new_product_df['Source'] = 'AdventureWorks'
print(new_product_df)


In [ ]:
# Voeg de ontbrekende kolommen toe aan elke DataFrame en orden ze in dezelfde volgorde
columns = ['NewProductID', 'ProductID', 'Source', 'ProductName', 'ProductDescription', 'ProductCategory', 'ProductColor', 'ProductSize', 'ProductQuantity', 
           'QuantityPerUnit', 'UnitPrice', 'ListPrice', 'UnitsInStock']

products_df = products_df.reindex(columns=columns)
product_df = product_df.reindex(columns=columns)
new_product_df = new_product_df.reindex(columns=columns)

# Combineer de dataframes
combined_df = pd.concat([products_df, product_df, new_product_df], ignore_index=True)

# Genereer een nieuwe primaire sleutel
combined_df['NewProductID'] = range(1, len(combined_df) + 1)

# Converteer null waarden naar 0.0 voor UnitPrice en ListPrice
combined_df['UnitPrice'] = combined_df['UnitPrice'].fillna(0.0).astype(float)
combined_df['ListPrice'] = combined_df['ListPrice'].fillna(0.0).astype(float)

# Zorg ervoor dat alle waarden correct zijn getypt
combined_df['NewProductID'] = combined_df['NewProductID'].astype(int)
combined_df['ProductID'] = combined_df['ProductID'].astype(int)
combined_df['Source'] = combined_df['Source'].astype(str)
combined_df['ProductName'] = combined_df['ProductName'].astype(str)
combined_df['ProductDescription'] = combined_df['ProductDescription'].astype(str)
combined_df['ProductCategory'] = combined_df['ProductCategory'].astype(str)
combined_df['ProductColor'] = combined_df['ProductColor'].astype(str)
combined_df['ProductSize'] = combined_df['ProductSize'].astype(str)
combined_df['ProductQuantity'] = combined_df['ProductQuantity'].fillna(0).astype(int)
combined_df['QuantityPerUnit'] = combined_df['QuantityPerUnit'].astype(str)
combined_df['UnitsInStock'] = combined_df['UnitsInStock'].fillna(0).astype(int)

print(combined_df)


In [ ]:
# Data laden in de nieuwe database (UnitedOutdoors)
for index, row in combined_df.iterrows():
    try:
        export_cursor.execute("""
        INSERT INTO Product (NewProductID, ProductID, Source, ProductName, ProductDescription, ProductCategory, ProductColor, ProductSize, ProductQuantity, QuantityPerUnit, UnitPrice, ListPrice, UnitsInStock)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, row.NewProductID, row.ProductID, row.Source, row.ProductName, row.ProductDescription, row.ProductCategory, row.ProductColor, row.ProductSize, row.ProductQuantity, row.QuantityPerUnit, row.UnitPrice, row.ListPrice, row.UnitsInStock)
    except pyodbc.Error as e:
        print(f"Error inserting row {index}: {e}")
        print(row)

export_conn.commit()
